<div>
    <img style="float:right;" src="images/smi-logo.png"/>
    <div style="float:left;color:#58288C;"><h1>Datenanalyse und Datenmanagement</h1></div>
</div>

---
# Notebook III: Data Exploration
In diesem Notebook geht es um die Verwendung von Python zur Erkundung eines eingelesenen Datenbestands.  
Hierzu werden einfache statistische Techniken und Visualisierungsformen eingesetzt.

## Inhaltsverzeichnis

[1. Daten einlesen und aufbereiten](#kapitel1)  
[2. Merkmale untersuchen und visualisieren](#kapitel2)  
[3. Zusammenhänge untersuchen und visualisieren](#kapitel3)  
[4. Anwendungsbeispiel Kreditvergabe](#kapitel4)  

---

## 1. Daten einlesen und aufbereiten <a id="kapitel1"/>

### 1.1. DataFrames und grobe Datenvalidierung
Pandas sind das zentrale Werkzeug zum einlesen und manipulieren von Daten in Python. Für unsere Zwecke ist die Datenstruktur des ```DataFrames``` am wichtigsten:

> **DataFrame** is a 2-dimensional labeled data structure with columns of potentially different types.  
> You can think of it like a spreadsheet or SQL table [...]. It is generally the most commonly used pandas object.  
> [(Quelle)](https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html)

Dataframes haben demnach Zeilen ("Rows") und Spalten ("Columns") können zudem einen Index haben - ein datensatzidentifizierendes Merkmal, ähnlich dem Primärschlüssel in der Datenbank.  
<img src="images/dataframe.png"/>  
DataFrames organisieren die Daten im Arbeitsspeicher, die mit SQL oder über APIs aus Datenquellen eingelesen werden. Die Dokumentation zu DataFrames findest du [hier](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html). 

Nutzen wir die Daten der COVID19-API aus der letzten Übung als Beispiel: 

In [ ]:
# Notebook extensions und Python Packages laden
%load_ext sql

import pandas as pd
import requests

In [ ]:
# API abfragen
response = requests.get("https://corona-api.com/timeline")
df = pd.json_normalize(response.json(), record_path="data")   # Im DataFrame df liegen nun die API-Daten

Nun sind die abgefragten Daten im DataFrame ```df```. Wir überprüfen zunächst oberflächlich die Fehlerfreiheit:

In [ ]:
df.head(5)   # die ersten fünf Zeilen anzeigen

In [ ]:
df.tail(5)   # die letzten drei Zeilen anzeigen

In [ ]:
df.count()   # Anzahl der Werte je Spalte prüfen - gibt es Lücken?

### 1.2. Spalten organisieren und Datentypen festlegen

> Wichtig: Bei allen Bearbeitungen des DataFrames wird eine Kopie mit den Änderungen erzeugt. Das ursprüngliche DataFrame bleibt unverändert.  
> Soll eine Änderung vorgenommen werden, kann der Ursprungsvariablen das geänderte DataFrame direkt zugeordnet werden.  
> _Beispiel: df = df.some_function()_

In [ ]:
# Bekommen wir aus der API auch unnötige Spalten, verwerfen wir diese zunächst
# (bei Datenbanken würden wir nur die benötigten Felder mit SQL abfragen)

# Die Funktion "drop" kann sowohl Zeilen als auch Spalten entfernen.
# Für Spalten ist eine Liste der Spaltennamen anzugeben, sowie der Verweis, dass es sich um Spalten handelt (axis="columns")

df = df.drop(["updated_at", "recovered", "new_recovered"], axis="columns")

# Dazu ist es oft praktisch, Spalten einheitlich oder kürzer zu benennen

df = df.rename(columns = {
    "new_confirmed": "new_cases", 
    "confirmed":"cases"
})

df.head(2)

In [ ]:
# Damit Datums- und Zeitangaben etwa bei der Sortierung korrekt behandelt werden können, ist im DataFrame der Datentyp noch korrekt zu setzen.
df.date = df.date.astype("datetime64")   # astype versucht den Inhalt des Strings 

# Nun kann das DataFrame das Datum interpretieren und wir können auf Tag/Monat/Jahr separat zugreifen
print(df.loc[0,"date"].year)
print(df.loc[0,"date"].month)
print(df.loc[0,"date"].day)

In [ ]:
# Das Datum ist bislang eine reguläre Spalte, ein vom DataFrame separat geführter Index zählt die Zeilen von 0-n durch.
# Da es sich hier um Meldezeitpunkte handelt, ist das Datum der Index. Damit wird bspw. der Index automatisch als X-Achse in späteren Grafiken verwendet.

df = df.set_index("date")
df.head(2)

### 1.3. Daten sortieren und filtern

In [ ]:
# Sortieren
df = df.sort_values("date", axis="rows", ascending=True)

In [ ]:
df.head(5)

#### Spaltenzugriff und Felder
Der Zugriff auf eine Spalte ist mit ```df["feldname"]``` möglich, wenn der Name keine Leer- oder Sonderzeichen enthält auch vereinfacht mit ```df.feldname```.

In [ ]:
# Um die Filterung zu verstehen sehen wir uns folgendes an:

df.new_cases # gibt die Spalte zurück

In [ ]:
df.new_cases > 300000 # gibt eine Spalte zurück, in denen jeweils das Ergebnis der Vergleichsoperation steht (True/False)

Um nun zu filtern, übergeben wir diesen True/False Spaltenvektor an das DataFrame, um die gewünschten Zeilen zu selektieren:

In [ ]:
peaks = df[df.new_cases > 300000]                                      # alle Tage an denen > 300k Neuinfektionen auftraten

peaks = peaks.sort_values("new_cases", axis="rows", ascending=False)   # absteigend nach Anzahl der Neuinfektionen sortieren
peaks

### 1.4. Berechnungen durchführen

In [ ]:
# Neue Spalte und Berechnungen

df["death_rate"] = df.new_deaths / df.active
df.death_rate                                   # der Index "date" wird automatisch mit angezeigt, wenn wir die neue Spalte abrufen

In [ ]:
# Konditionale Zuweisungen
df["bad_day"] = 0                                  # Spalte anlegen, Wert ist überall 0
df.loc[df.new_cases > 300000, "bad_day"] = 1       # Werte auf 1 setzen, wenn new_cases > 300.000

print("An {} von {} Tagen sind mehr als 300.000 neue Fälle aufgetreten.".format(df.bad_day.sum(), df.bad_day.count()))

### 1.5. DataFrame partitionieren

In [ ]:
# Mit DataFrames kann man ähnlich Listen und Dictionaries arbeiten

print("Zugriff über Index")
print(df.iloc[0,1])
print("Zugriff über Indexausprägung und Spaltennamen")
print(df.loc["2020-01-21","cases"])

In [ ]:
print("Ranges")
print(df.loc["2020-02-01":"2020-02-14", "deaths":"active"])

In [ ]:
print("Subsets")
print(df.loc["2020-02-01", ["deaths", "active"]])

---
## 2. Merkmale untersuchen und visualisieren <a id="kapitel2"/>

### 2.1. Deskriptive Statistiken

In [ ]:
# Deskriptive Statistiken können für das gesamte Dataframe oder einzelne Spalten berechnet werden

# df.describe()
df.new_cases.describe()

In [ ]:
# Für jede Spalte können mit analoger Syntax die wesentlichen statistischen Maße berechnet werden.
print("Median, Mean, Max, Min")
print(df.cases.median(), df.cases.mean(), df.cases.max(), df.cases.min())

In [ ]:
# Quantile

p20 = df.new_cases.quantile(0.2)
print("An 20% der Tage gab es weniger als {} Neuinfektionen.".format(p20))

### 2.2. Visualisierung mit Matplotlib

Pandas haben eine plot() Funktion, die einfache Grafiken erzeugt. Die Dokumentation der ```plot```-Funktion findest du [hier](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.plot.html).

In [ ]:
# Create plots

# Jupyter-Feature: Wir wollen das Diagramm direkt im Notebook sehen, nicht als Datei speichern o.ä.
%matplotlib inline

df.death_rate.plot()

In [ ]:
# Boxplots und ausreißer bereinigen über Filter

df.new_cases.plot()

In [ ]:
df[df.index.month==8].new_cases.plot(kind="box")

In [ ]:
df[(df.new_cases==0) | (df.new_cases > 400000)].head(5)

In [ ]:
df = df[df.index!="2020-08-17"]
df = df[df.index!="2020-08-18"]

In [ ]:
# Neue Cases der letzten 14 Tage

df[-15:-1].new_cases.plot(kind="bar", title="Neuinfektionen der letzten 14 Tage")  

In [ ]:
# Histogramm der neuen Fälle

df.new_cases.plot(kind="hist") 

In [ ]:
# Vergleich der neuen Fälle vor/nach Juli mit Histogrammen

df.loc[df.index < "2020-07-01", "new_cases"].plot(kind="hist", alpha=0.5) 
df.loc[df.index >= "2020-07-01", "new_cases"].plot(kind="hist", alpha=0.5) 

---
## 3. Zusammenhänge untersuchen und visualisieren <a id="kapitel3"/>

Im ersten Beispieldatensatz sind vermutlich wenige Zusammenhänge zu interessant, allenfalls zwischen new_deaths und new_cases. Wir verwenden das umfassendere Visualisierungspaket Seaborn, um komplexe Grafiken mit einfachen Befehlen zu erzeugen.

In [ ]:
# Korrelationsmatrix
df.corr()

In [ ]:
# Advanced Styling mit Package Seaborn, Beispiele: https://seaborn.pydata.org/tutorial/distributions.html
import seaborn as sns

sns.heatmap(df.corr(), cmap="coolwarm")

In [ ]:
sns.pairplot(df.loc[:,["new_cases","new_deaths", "active"]])

---
## 4. Anwendungsbeispiel Kreditvergabe <a id="kapitel4"/>

Wir betrachten im Folgenden einen Datensatz mit Privatkreditvorgängen einer deutschen Bank.  
Hierbei durchlaufen wir dieselbe Prozedur, bauen aber die Grafiken etwas eleganter auf.

Der Datensatz kommt vom Datebankserver und umfasst folgende Merkmalen:
   >  *Age* (numeric)  
   >  *Sex* (text: male, female)  
   >  *Job* (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)  
   >  *Housing* (text: own, rent, or free)  
   >  *Saving accounts* (text - little, moderate, quite rich, rich)  
   >  *Checking account* (text - little, moderate, quite rich, rich)   
   >  *Credit amount* (numeric, in EUR)  
   >  *Duration* (numeric, in month)  
   >  *Purpose* (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others  
   >  *Risk* (Value target - Good or Bad Risk)  

### 4.1. Daten einlesen und aufbereiten

In [ ]:
# Module importieren

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Visualisierung-Stil für Diagramme mit Seaview setzen
sns.set(font_scale = 1.1,               # Schriften etwas größer als normal
       palette     = "pastel",        # siehe: http://seaborn.pydata.org/tutorial/color_palettes.html
       style       = "whitegrid")       # Schriften etwas größer als normal

In [ ]:
# Verbindung zum Datenbankserver herstellen

%load_ext sql
%sql sqlite:///data/smi-data.db

# SQL-Abfrage durchführen und Ergebnis in Variable result speichern        
        
result = %sql SELECT * FROM credit_ger

# Aus Result ein DataFrame machen und in Variable df speichern

df = result.DataFrame()
df = df.set_index(["id"])
df.head(5)

In [ ]:
# Spaltenbenennungen vereinfachen
df = df.rename({
    "Age": "age",
    "Sex": "sex",
    "Job": "job",
    "Housing": "housing",
    "Saving accounts": "savings", 
    "Checking account": "cash",
    "Credit amount": "amount",
    "Duration": "duration",
    "Purpose": "purpose",
    "Risk": "risk"
}, axis="columns")

### 4.2. Merkmale untersuchen und visualisieren

In [ ]:
# Deskriptive Statistik der wichtigsten Merkmale

df[['age', 'duration', 'amount']].describe()

#### Untersuchung Kreditausfälle und Alter

In [ ]:
# Globale Kreditausfallquote

# Berechnung mit DataFrame-Filtern

credit_default_rate = df[df.risk=="good"].risk.count() / df.risk.count()
print (credit_default_rate)

In [ ]:
# Alternative Berechnung mit Kreuztabelle

pd.crosstab(df.risk, columns="count", normalize="columns")      # normalize: keine abs. Häufigkeit, sondern relative Häufigkeit in Spalten angeben

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16,6))
fig.suptitle("Kreditausfälle und Alter")

ax[0].set_title("Kreditausfälle")
sns.histplot(df.risk, ax=ax[0])

ax[1].set_title("Altersverteilung")
sns.histplot(df.age, ax=ax[1])

ax[2].set_title("Altersverteilung nach Kreditauswall")
sns.histplot(x=df.age, hue=df.risk, ax=ax[2], palette="RdYlGn_r", bins=20)

plt.show()

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(20,10))
plt.suptitle("Verteilungen")
sns.histplot(df.amount, bins=40, ax=ax[0,0]);
sns.histplot(df.duration, bins=40, ax=ax[0,1], color="salmon");
sns.histplot(df.age, bins=40, ax=ax[0,2], color="darkviolet");

sns.boxplot(x=df.amount, ax=ax[1,0]);
sns.boxplot(x=df.duration, ax=ax[1,1], color="salmon");
sns.boxplot(x=df.age, ax=ax[1,2], color="darkviolet");

plt.show() 

#### Auffälligkeiten
- Kredite typischerweise 1400-4000 EUR und 1-2 Jahre
- Kredite mit vielen Ausreißerun nach oben (>7500)
- Kreditnehmer Ende 20 bis Anfang 40 

### 4.3. Zusammenhänge untersuchen und visualisieren

In [ ]:
# Bestehen Zusammenhänge zwischen den kardinalen und/oder metrischen Merkmalen?

sns.pairplot(df, hue="risk")

#### Auffälligkeiten
- Amount vs. Duration: Größere Kredite werden in überwiegender Zahl für längere Laufzeiten vergeben.

In [ ]:
# Zweck des Kredits vs. Rückzahlungshäufigkeit

pd.crosstab(df.purpose, df.risk, normalize="index")        # normalize: keine absolute Häufigkeit sondern zeilenweise (index) relative Häufigkeit angeben

In [ ]:
# Rückzahlungsquoten scheinen bei 60-70% zu liegen
# Genauere Untersuchung durch Visualisierung
fig, ax = plt.subplots(1,2,figsize=(18,8))
plt.suptitle('Zweck des Kredits')

sns.countplot(y=df.purpose, hue=df.risk, ax=ax[0], palette="RdYlGn_r")
sns.boxplot(y=df.purpose, x=df.amount, hue=df.risk, ax=ax[1], palette="RdYlGn_r")

plt.show()

#### Auffälligkeiten
- Hoher Zusammenhang zwischen Ausfallrisko und Kreditzweck
- Übermäßig teure Reisekredite (>11k) fallen alle aus
- Höhere Kredite fallen eher aus

In [ ]:
# Zweck des Kredits vs. Wohnsituation und Qualifikation

fig, ax = plt.subplots(1,1,figsize=(18,8))
plt.suptitle('Housing und Qualifikation')

sns.violinplot(x=df.housing, y=df.job, hue=df.risk, palette="RdYlGn_r");

#### Auffälligkeiten
- Geringe Qualifikation, kein Wohneigentum oder Mietverhältnis ist eine problematische Kombination
- Bei Wohneigentum generell höhere Rückzahlungshäufigkeit, insbl. mit Berufsausbildung ("skilled", job = 2)
- In höchster Qualifikationsstufe keine Wechselwirkung mit Wohnsituation erkennbar

---
### <span style="color:#FF5D02;">Aufgabe 1: Exploration</span>
Untersuche die Merkmale ```sex```, ```savings``` und ```cash``` einzeln und im Zusammenhang mit der Kreditrückzahlungsquote!
Führe die Untersuchung sowohl tabellarisch als auch grafisch durch.